In [ ]:
import pyabc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from koulakov_based_model import DH_ST_Model
from pyabc_koulakov import model_all_conditions, distance_all_conditions_full, PRIORS_ALL_CONDITIONS

abc = pyabc.ABCSMC(
    models = model_all_conditions,
    distance_function=distance_all_conditions_full,
    parameter_priors=PRIORS_ALL_CONDITIONS,
    transitions=pyabc.transition.LocalTransition()
)

db_path = f"pyabc_runs/pyabc_output.db"
history = abc.load("sqlite:///" + db_path)
df, weights = history.get_distribution(m=0, t=history.max_t)
epsilons = np.array(history.get_weighted_distances()["distance"])
parameters = df.iloc[epsilons.argmin()].to_dict()

In [ ]:
# normal
model = DH_ST_Model(
    dh_dims=(50, 1),
    steps_per_axon=500,
    **parameters
)
model.run()
model.energy_history_plot()
model.n_synapse_history_plot()
model.correlation_plots()

In [ ]:
# disrupted activity
parameters_disrupted = parameters.copy()
parameters_disrupted["gamma"] = parameters["gamma"] * 0.20  # assume 80% reduction in activity-dependent refinement

model = DH_ST_Model(
    dh_dims=(50, 1),
    steps_per_axon=500,
    **parameters_disrupted
)
model.run()
model.energy_history_plot()
model.n_synapse_history_plot()
model.correlation_plots()